In [1]:
import os
os.chdir(r'C:\Users\Navid\Documents\ds_salary_proj')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [42]:
df = pd.read_csv('eda_data_modified')

In [43]:
df['seniority'].value_counts()

0    1108
1     303
Name: seniority, dtype: int64

In [44]:
# df = df.loc[df.Industry != '-1']
df = df.loc[df['Type of ownership'] != 'Unknown']

In [45]:
df = df.loc[(df.Size != '-1') & (df.Size != 'Unknown')]

In [46]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Job Description', 'Rating', 'Company Name',
       'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'Competitors', 'per_hour',
       'avg_salary', 'FAANG', 'state', 'desc_length', 'seniority', 'title',
       'python', 'r studio', 'ML', 'spark', 'deep_learning', 'aws', 'graduate',
       'undergrad', 'exp_year'],
      dtype='object')

In [47]:
df_final = df[['avg_salary' ,'Size','Type of ownership', 'Rating', 'Industry',
               'Sector', 'Revenue','per_hour', 'seniority', 
               'state', 'aws', 'title', 'spark', 'FAANG',
       'deep_learning', 'graduate', 'undergrad','exp_year']]

In [48]:
df = df[df.Rating != -1]

In [49]:
#removing skewness
# from scipy import stats
# df['avg_salary'] ,fitted_lambda1 = stats.boxcox(df['avg_salary'])

In [50]:
df_dummy = pd.get_dummies(df_final)

In [51]:
# df_dummy.drop('Industry_other_industries',axis=1,inplace=True)

In [52]:
# train test split
from sklearn.model_selection import train_test_split
x = df_dummy.drop('avg_salary' , axis = 1)
y = df_dummy['avg_salary'].values
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 1) 

In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
lr = SVR(C=10).fit(X_train,y_train)
cross_val_score(lr,X_train,y_train,scoring='neg_mean_absolute_error',cv=5).mean()

-14.954116466524573

In [55]:
import statsmodels.api as sm
X_sm = sm.add_constant(x)
model = sm.OLS(y,X_sm)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     19.76
Date:                Sat, 31 Oct 2020   Prob (F-statistic):          3.22e-227
Time:                        18:40:48   Log-Likelihood:                -6141.4
No. Observations:                1404   AIC:                         1.255e+04
Df Residuals:                    1271   BIC:                         1.325e+04
Df Model:                         132                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
const                                               41.0079      3.726     11.006      0.000      33.698      48.318
Rating                                               3.6768      1.115      3.296      0.001       1.489       5.865
per_hour                                           -14.5339     10.589     -1.373      0.170     -35.308       6.240
seniority                                           23.2481      1.474     15.777      0.000      20.357      26.139
aws                                                  1.8748      1.368      1.370      0.171      -0.810       4.559
spark                                                0.0966      1.935      0.050      0.960      -3.700       3.893
FAANG                                               12.0738      7.614      1.586      0.113      -2.864      27.012
deep_learning                                        1.1018      2.634      0.418      0.676      -4.065       6.269
graduate                                             2.2704      1.293      1.757      0.079      -0.265       4.806
undergrad                                           -4.3159      1.256     -3.437      0.001      -6.779      -1.853
exp_year                                             0.8725      0.250      3.489      0.001       0.382       1.363
Size_1 to 50 Employees                               2.9547      3.413      0.866      0.387      -3.741       9.650
Size_10000+ Employees                                6.9689      2.126      3.277      0.001       2.797      11.140
Size_1001 to 5000 Employees                          6.6475      1.468      4.528      0.000       3.767       9.528
Size_201 to 500 Employees                            5.1574      1.728      2.984      0.003       1.766       8.548
Size_5001 to 10000 Employees                         7.8115      2.284      3.420      0.001       3.330      12.293
Size_501 to 1000 Employees                           6.8987      1.732      3.983      0.000       3.501      10.297
Size_51 to 200 Employees                             4.5693      1.916      2.385      0.017       0.811       8.327
Type of ownership_College / University              -0.5487      9.130     -0.060      0.952     -18.460      17.363
Type of ownership_Company - Private                  6.9963      3.055      2.290      0.022       1.003      12.989
Type of ownership_Company - Public                   7.0313      3.198      2.199      0.028       0.758      13.304
Type of ownership_Contract                           1.6323     11.655      0.140      0.889     -21.233      24.498
Type of ownership_Government                         2.5409      5.663      0.449      0.654      -8.568      13.650
Type of ownership_Hospital               

In [56]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
grid_param = {'n_estimators' : range(200,300,20)}
grid = GridSearchCV(RandomForestRegressor(n_jobs=-1 ,random_state = 0), param_grid = grid_param, scoring = 'neg_mean_absolute_error',  cv =3).fit(X_train , y_train)



In [57]:
print(grid.best_score_,grid.best_estimator_)

-13.997687569444485 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=280, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)


In [80]:
df_dummy.corr()['avg_salary'].sort_values(ascending =False)[0:40]

avg_salary                                          1.000000
title_data scientist                                0.458742
seniority                                           0.381329
state_CA                                            0.311553
deep_learning                                       0.226122
spark                                               0.222998
phd                                                 0.216100
exp_year                                            0.215850
aws                                                 0.205451
Industry_Internet                                   0.176393
Sector_Information Technology                       0.169561
title_data engineer                                 0.163343
title_machine learning engineer                     0.160460
Rating                                              0.152159
Industry_Biotech & Pharmaceuticals                  0.150314
Sector_Biotech & Pharmaceuticals                    0.150314
title_data architect    

In [20]:
grid.best_estimator_.score(X_test,y_test)

0.5771392144944245

In [82]:
df[df['Company Name'].duplicated().values == True].sort_values('Company Name')

,Unnamed: 0,Unnamed: 0.1,Job Title,Salary Range,Job Description,Rating,Company Name,Location,Headquarters,Size,...,python,r studio,ML,spark,deep_learning,aws,phd,undergrad,master,exp_year
1093,1484,62,Data Scientist,$89k-$153k,The Data Scientist will be joining a team of e...,4.1,7Park Data,"New York, NY",-1,51 to 200 Employees,...,1,0,1,1,0,0,1,0,1,5.0
892,1250,84,Marketing Data Analyst,$70k-$100k,Were seeking a Marketing Data Analyst with an ...,4.6,Ability Network,"Santa Monica, CA",-1,51 to 200 Employees,...,0,0,0,0,0,1,0,0,0,5.0
1232,1743,111,Data Analyst,$40k-$78k,Company Overview:\nAge of Learning is a leadin...,3.1,Age of Learning,"Glendale, CA",-1,501 to 1000 Employees,...,0,0,1,0,0,0,0,1,0,2.0
763,1087,432,Senior Data Scientist,$110k-$176k,Company Overview:\nAge of Learning is a leadin...,3.2,Age of Learning,"Glendale, CA",-1,501 to 1000 Employees,...,1,0,1,0,0,0,0,1,1,4.0
504,705,88,Data Scientist,$80k-$132k\r,Job Capsule\r\n\r\n\r\nThe Data Scientist anal...,3.4,American Bureau of Shipping\r,"Houston, TX",-1,1001 to 5000 Employees,...,1,0,1,0,1,0,0,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,969,271,Lead Data Engineer,$47k-$93k,"Data Analyst\nLocation: Washington, DC\nEducat...",4.0,Zipcar,"Boston, MA",-1,501 to 1000 Employees,...,0,0,0,0,0,0,0,1,1,3.0
1212,1712,151,Lead Data Engineer,$83k-$147k,Lead Data Engineer\n\nAs a data engineer in Zi...,3.2,Zipcar,"Boston, MA",-1,201 to 500 Employees,...,1,0,0,1,0,1,0,1,0,5.0
831,1171,304,Sr Data Science Analyst,$61k-$107k,Job Summary\nThe Sr. Data Science Analyst oper...,3.6,"naviHealth, Inc.","Brentwood, TN",-1,1001 to 5000 Employees,...,1,0,1,1,1,0,0,1,0,3.5
670,964,264,Staff Data Scientist,$123k-$196k,"At realtor.com, we process terabytes of data e...",2.8,realtor.com,"Santa Clara, CA",-1,1001 to 5000 Employees,...,1,0,1,1,0,0,1,0,0,4.0


In [104]:
df_final.shape

(718, 21)

In [105]:
df.title.value_counts()

data analyst                 385
data scientist               214
data engineer                 93
machine learning engineer     18
data architect                 8
Name: title, dtype: int64

In [110]:
df.loc[807]

Unnamed: 0                                                        1012
Unnamed: 0.1                                                        63
Job Title                                                Data Engineer
Salary Range                                                $90k-$135k
Job Description      About Us\n\n\nInterested in working for a huma...
Rating                                                             4.7
Company Name                                                  1904labs
Location                                               Saint Louis, MO
Headquarters                                                        -1
Size                                               51 to 200 Employees
Founded                                                              4
Type of ownership                                    Company - Private
Industry                                                   IT Services
Sector                                          Information Technology
Revenu